# Предсказательные модели для Dota 2

Проект разработки сервиса, который по ID матча выдает вероятность победы той или иной команды в Dota 2, а также строит различные сегментационные карты (хитмапы) по результатам игры.

**Участники:**
* Боева Полина Викторовна — @Boeva_pv, PolinaBoeva
* Бородачев Сергей Игоревич — @RaiFox, Rai-Fox
* Герилович Илья Павлович — @mrbrainers, ilyager99
* Чаплыгина Арина Сергеевна — @chaarse, chaarse

**Куратор:** Макарова Мария - @mariagolddd

In [35]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

## Шаг 1. Сбор данных

In [ ]:
match = {}
for i in range(1, 10):
    key = f'm{i}'
    file_path = f'/kaggle/input/dota-2-pro-league-matches-2023/20240{i}/main_metadata.csv'
    match[key] = pd.read_csv(file_path)

key = 'd10'
file_path = '/kaggle/input/dota-2-pro-league-matches-2023/202410/main_metadata.csv'
match[key] = pd.read_csv(file_path)


players = {}
for i in range(1, 10):
    key = f'p{i}'
    file_path = f'/kaggle/input/dota-2-pro-league-matches-2023/20240{i}/players.csv'
    players[key] = pd.read_csv(file_path, dtype='str')

key = 'd10'
file_path = '/kaggle/input/dota-2-pro-league-matches-2023/202410/players.csv'
players[key] = pd.read_csv(file_path)

In [ ]:
match_df = pd.concat(match.values(), ignore_index=True)
players_df = pd.concat(players.values(), ignore_index=True)

In [ ]:
match_df.drop(columns=['Unnamed: 0'], inplace=True)
players_df.drop(columns=['Unnamed: 0'], inplace=True)

In [ ]:
match_df.head()

In [ ]:
players_df.head()

## Шаг 2. Разведочный анализ данных

### Датасет *match_df*

In [ ]:
match_df.info()

In [ ]:
match_df.dropna(axis=1, how='all', inplace=True)

In [ ]:
match_df.isna().sum()

In [ ]:
n_rows = len(match_df)
cols_with_na = match_df.columns[match_df.isna().any()] # столбцы с пропущенными значениями

plt.figure(figsize=(10, 6))
sorted_cols = match_df.isna().sum()[cols_with_na].sort_values(ascending=False).index

ax = sns.barplot(y=sorted_cols, x=match_df.isna().sum()[sorted_cols], orient='h', color="skyblue")

for i, v in enumerate(match_df.isna().sum()[sorted_cols]):
    ax.text(v + 50, i, f"{v / n_rows:.2%}", va='center', fontsize=10)

plt.xlabel('Количество пропущенных значений')
plt.ylabel('Столбцы')
plt.title('Пропущенные значения по столбцам')
plt.show()

### Обработка пропусков

In [ ]:
# удалим строки с пропусками, которые составляют менее 5%
threshold = 0.05
index = match_df.isna().mean() < threshold
columns_to_drop = index[index == True].index

match_df = match_df.dropna(subset=columns_to_drop)

In [ ]:
# столбцы, в которых содержится более 5% пропусков
match_df[['series_id', 'series_type', 'throw', 'loss', 'comeback', 'stomp', 'radiant_logo', 'dire_logo']]

Описание столбцов с пропусками:

* series_id (идентификатор серии матчей): возможно, информация о серии матчей не всегда фиксировалась или была доступна только для части матчей. Пропущенные значения можно заменить на модальное значение.
* series_type (тип серии матча): аналогично series_id
* throw (признак того, что одна из команд сознательно проиграла матч): по умолчанию можно считать, что если информация о бросании отсутствует, значит, команда не бросала игру. Никак не заменяем.
* loss (признак поражения одной из команд): аналогично throw.Никак не заменяем.
* comeback (признак камбека в матче): если информация о камбеке отсутствует, можно считать его не произошедшим и никак не изменять.
* stomp (признак доминирования одной из команд над другой): аналогично comeback. Так же, как и для предыдущих полей, можно никак не изменять.
* radiant_logo (логотип команды Radiant): логотип мог быть недоступен или не загружен на момент сохранения данных. Никак не заменяем.
* dire_logo (логотип команды Dire): причины те же, что и для radiant_logo. Никак не заменяем.

In [ ]:
fill_values = {
    'series_id': match_df['series_id'].mode()[0],
    'series_type': match_df['series_type'].mode()[0]
}

match_df.fillna(fill_values, inplace=True)
match_df.isna().sum()

### Расчет основных статистик

In [ ]:
match_df.describe(include='all')

### Описательная статистика

In [ ]:
num_match_df = match_df[['match_id', 'cluster', 'dire_score', 'duration', 'game_mode', 
                        'human_players', 'leagueid', 'lobby_type', 'match_seq_num', 'radiant_score', 
                        'radiant_win', 'version', 'replay_salt', 'series_id', 'series_type', 
                        'patch', 'region', 'throw', 'loss', 'comeback', 'stomp', 'dire_team_id', 
                        'radiant_team_id', 'pre_game_duration', 'flags', 'radiant_logo', 
                        'radiant_team_complete', 'dire_logo', 'dire_team_complete', 
                        'radiant_captain', 'dire_captain']]
corr_matrix = num_match_df.corr()
corr_matrix

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
mask = np.zeros_like(corr_matrix, dtype=bool)
mask[np.triu(corr_matrix) != 0] = True
sns.heatmap(data=corr_matrix, annot=True, cmap='coolwarm', center=0, square=True, cbar_kws={"shrink": 0.75}, fmt='.2f', annot_kws={'size': 6})
plt.title('Матрица корреляционного анализа')
plt.tight_layout()
plt.show()

In [ ]:
target = 'radiant_win'
correlations = abs(corr_matrix[target])
top_correlations = correlations.sort_values(ascending=False)
top_correlations[:10]

In [ ]:
mean_duration = match_df.groupby('radiant_win')['duration'].mean()
mean_duration.plot(kind='bar', color=['red', 'blue'], figsize=(8, 5))
plt.title('Средняя продолжительность матча по победам')
plt.xlabel('Результат')
plt.ylabel('Средняя продолжительность (в секундах)')
plt.xticks([0, 1], ['Dire Win', 'Radiant Win'], rotation=0)
plt.show()

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

sns.histplot(data=match_df, x='radiant_score', hue='radiant_win', multiple='stack', 
             kde=True, palette='light:b', bins=30)
plt.title('Распределение очков Radiant')
plt.xlabel('Очки Radiant')
plt.ylabel('Количество матчей')
plt.show()

In [ ]:
region_wins = match_df.groupby(['region', 'radiant_win']).size().unstack()
region_wins.plot(kind='bar', stacked=True, figsize=(12, 6))
plt.title('Победы по регионам')
plt.xlabel('Регион')
plt.ylabel('Количество матчей')
plt.legend(title='Результат')
plt.show()

### Оставим в датасете только те столбцы, которые могут влиять на результат матча:

In [ ]:
match_df = match_df[['match_id', 'barracks_status_dire', 'barracks_status_radiant', 'dire_score',
                     'radiant_score', 'duration', 'first_blood_time', 'game_mode', 'human_players',
                     'lobby_type', 'radiant_win', 'tower_status_dire', 'tower_status_radiant', 
                     'throw', 'loss', 'comeback', 'stomp', 'pre_game_duration']]
match_df.head()

In [ ]:
win_counts = match_df['radiant_win'].value_counts()
win_counts.plot(kind='bar', color=['blue', 'red'])
plt.title('Распределение побед Radiant и Dire')
plt.xlabel('Победа')
plt.ylabel('Количество матчей')
plt.xticks(ticks=[0, 1], labels=['Dire', 'Radiant'], rotation=0)
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(x='radiant_win', y='duration', data=match_df)
plt.title('Продолжительность матчей в зависимости от победы')
plt.xlabel('Победа (0 - Dire, 1 - Radiant)')
plt.ylabel('Продолжительность (в секундах)')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(match_df['first_blood_time'], bins=30, kde=True)
plt.title('Распределение времени первой крови')
plt.xlabel('Время первой крови (в секундах)')
plt.ylabel('Количество матчей')
plt.show()

In [ ]:
sns.set(style="whitegrid")

for column in match_df.columns:
    plt.figure(figsize=(10, 6))
    
    if pd.api.types.is_numeric_dtype(match_df[column]):
        sns.histplot(match_df[column], bins=30, kde=False)
        plt.title(f'Гистограмма для {column}')
        plt.xlabel(column)
        plt.ylabel('Количество')
    else:
        value_counts = match_df[column].value_counts()
        sns.barplot(x=value_counts.index.astype(str), y=value_counts.values)
        plt.title(f'Bar Plot для {column}')
        plt.xlabel(column)
        plt.ylabel('Количество')
        plt.xticks(rotation=45)
    
    plt.show()


In [ ]:
match_df = match_df[match_df['game_mode'] == 2]
match_df = match_df[match_df['human_players'] == 10]
match_df = match_df[match_df['lobby_type'] == 1]

### Почистим датасет *players_df* и объединим с *match_df*

In [ ]:
players_df.columns.to_list()

In [ ]:
selected_columns = [
    'match_id',
    'duration',
    'game_mode',
    'start_time',
    'radiant_win',
    'hero_id',
    'player_slot',
    'kills',
    'deaths',
    'assists',
    'gold_per_min',
    'xp_per_min',
    'last_hits',
    'denies',
    'level',
    'hero_damage',
    'item_0',
    'item_1',
    'item_2',
    'item_3',
    'item_4',
    'item_5'
]

players_df = players_df[selected_columns]
players_df.head()

In [ ]:
players_df.describe(include='all')

In [ ]:
players_df['radiant_win'].unique()

In [ ]:
players_df = players_df[players_df['radiant_win'].isin([True, False])]
players_df = players_df[players_df['game_mode'] == 2]

In [ ]:
players_df.isna().sum()

In [ ]:
merged_df = pd.merge(match_df, players_df, on='match_id', how='inner')
merged_df.head()

In [ ]:
merged_df.columns

# Описание полей в Dota 2 датасете

| Поле                        | Тип       | Описание                                                                                  |
|-----------------------------|-----------|-------------------------------------------------------------------------------------------|
| `match_id`                  | Integer   | Уникальный идентификатор матча.                                                           |
| `barracks_status_dire`      | Integer   | Состояние бараков команды Dire (0 - все живые, 1 - один разрушен, 2 - оба разрушены).   |
| `barracks_status_radiant`   | Integer   | Состояние бараков команды Radiant (0 - все живые, 1 - один разрушен, 2 - оба разрушены).|
| `dire_score`                | Integer   | Очки команды Dire по окончании матча.                                                    |
| `radiant_score`             | Integer   | Очки команды Radiant по окончании матча.                                                |
| `duration_x`                | Integer   | Продолжительность матча в секундах (версия X).                                          |
| `first_blood_time`          | Integer   | Время, когда произошло первое убийство (в секундах).                                     |
| `game_mode_x`               | Integer   | Режим игры (например, 1 для "All Pick", 2 для "Captain's Mode" и т.д., версия X).     |
| `human_players`             | Integer   | Общее количество игроков-людей в игре.                                                   |
| `lobby_type`                | Integer   | Тип лобби (например, 0 для обычного, 1 для рейтингового и т.д.).                       |
| `radiant_win_x`             | Boolean   | Указывает, выиграла ли команда Radiant в версии X (True или False).                     |
| `tower_status_dire`         | Integer   | Состояние башен команды Dire (0 - все живые, 1 - одна разрушена, 2 - две разрушены и т.д.). |
| `tower_status_radiant`      | Integer   | Состояние башен команды Radiant (0 - все живые, 1 - одна разрушена, 2 - две разрушены и т.д.). |
| `throw`                     | Boolean   | Указывает, произошла ли "потеря" преимущества.                                           |
| `loss`                      | Boolean   | Указывает, проиграла ли команда (True - проиграла, False - выиграла).                   |
| `comeback`                  | Boolean   | Указывает, произошло ли "возвращение" команды после отставания.                         |
| `stomp`                     | Boolean   | Указывает, был ли матч "накатанным" (одна команда доминировала).                        |
| `pre_game_duration`         | Integer   | Продолжительность подготовки перед матчем в секундах.                                    |
| `duration_y`                | Integer   | Продолжительность матча в секундах (версия Y).                                          |
| `game_mode_y`               | Integer   | Режим игры (версия Y).                                                                    |
| `start_time`                | Integer   | Время начала матча в формате UNIX timestamp.                                             |
| `radiant_win_y`             | Boolean   | Указывает, выиграла ли команда Radiant в версии Y (True или False).                     |
| `hero_id`                   | Integer   | Уникальный идентификатор героя, выбранного игроком.                                     |
| `player_slot`               | Integer   | Номер слота игрока (0-4 для Radiant и 128-132 для Dire).                               |
| `kills`                     | Integer   | Общее количество убийств, совершенных игроком в матче.                                   |
| `deaths`                    | Integer   | Общее количество смертей игрока в матче.                                                 |
| `assists`                   | Integer   | Общее количество помощей, оказанных игроком в матче.                                     |кретной задачи.
